In [22]:
import tensorflow as tf
import os
import glob
import numpy as np
import scipy.signal

input_chessboard_folder = "./jinchess"
img_files = set(glob.glob("%s/*.png" % input_chessboard_folder))\
    .union(set(glob.glob("%s/*.jpg" % input_chessboard_folder)))\
    .union(set(glob.glob("%s/*.gif" % input_chessboard_folder)))
    
img_path = img_files.pop()

# Strip to just filename
img_file = img_path[len(input_chessboard_folder)+1:-4]

# Load image
print("---")
print("Loading %s..." % img_path)

def get_grayscale_image(img_file):
    """given a file like object of an image (not gif), return a grayscale float32, 2d tensor"""
    gs = tf.image.decode_png(img_file.read(), channels=1, name="grayscaleimage")
    shrunken = tf.squeeze(gs, name="shrunkenimage")
    result = tf.cast(shrunken, dtype=tf.float32, name="2dgrayscaleimage")
    result.set_shape([None, None])
    return result

def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return tf.constant(a, dtype=1)

def simple_conv(x, k):
  """A simplified 2D convolution operation"""
  x = tf.expand_dims(tf.expand_dims(x, 0), -1)
  y = tf.nn.depthwise_conv2d(x, k, [1, 1, 1, 1], padding='SAME')
  return y[0, :, :, 0]

def gradientx(x):
  """Compute the x gradient of an array"""
  gradient_x = make_kernel([[-1.,0., 1.],
                            [-1.,0., 1.],
                            [-1.,0., 1.]])
  return simple_conv(x, gradient_x)

def gradienty(x):
  """Compute the x gradient of an array"""
  gradient_y = make_kernel([[-1., -1, -1],[0.,0,0], [1., 1, 1]])
  return simple_conv(x, gradient_y)

gausswin = tf.constant(scipy.signal.gaussian(21,4), dtype=tf.float32, name="gausswin")
gausswin_sum = tf.reduce_sum(gausswin, name="gausswin_sum")
gausswin_pct = gausswin / gausswin_sum

def numpy_convolve(a, v, mode):
    """perform np.convolve in tensorflow"""
    # Assuming a.size = [x], cast to tensor of size [1,x,1] because tensorflow
    a_expanded = tf.expand_dims(tf.expand_dims(a, -1), 0)
    # likewise, v of [a, b, c] needs to be reversed and expanded to [[[c]], [[b]], [[a]]]
    v_expanded = tf.expand_dims(tf.expand_dims(tf.reverse(v, [0]), -1), -1)
    
    return tf.squeeze(tf.nn.conv1d(a_expanded, v_expanded,1, mode.upper()))

img_gray = get_grayscale_image(open(img_path,'rb'))

Dx = gradientx(img_gray)
Dy = gradienty(img_gray)

Dx_pos = tf.clip_by_value(Dx, 0., 255., name="dx_positive")
Dx_neg = tf.clip_by_value(Dx, -255., 0., name='dx_negative')
Dy_pos = tf.clip_by_value(Dy, 0., 255., name="dy_positive")
Dy_neg = tf.clip_by_value(Dy, -255., 0., name='dy_negative')


# 1-D ampltitude of hough transform of gradients about X & Y axes
# Chessboard lines have strong positive and negative gradients within an axis
img_shape = tf.cast(tf.shape(img_gray), dtype=tf.float32)
hough_Dx = tf.reduce_sum(Dx_pos, 0) * tf.reduce_sum(-Dx_neg, 0) / (img_shape[0]*img_shape[0])
hough_Dy = tf.reduce_sum(Dy_pos, 1) * tf.reduce_sum(-Dy_neg, 1) / (img_shape[1]*img_shape[1])

# Slightly closer to 3/5 threshold, since they're such strong responses
hough_Dx_thresh = tf.reduce_max(hough_Dx) * 3/5
hough_Dy_thresh = tf.reduce_max(hough_Dy) * 3/5

# start getChessLines
blur_x_tf = numpy_convolve(tf.cast(hough_Dx > hough_Dx_thresh, dtype=tf.float32), gausswin_pct, mode='same')
blur_y_tf = numpy_convolve(tf.cast(hough_Dy > hough_Dy_thresh, dtype=tf.float32), gausswin_pct, mode='same')

#start 

---
Loading ./jinchess\jinchess155.rnbqkbnrpppppppp---kq----------------------KQ---PPPPPPPPRNBQKBNR.png...


In [23]:
def get_slope(prev, cur):
    # A: Ascending
    # D: Descending
    # P: PEAK (on previous node)
    # V: VALLEY (on previous node)
    return tf.cond(prev[0] < cur, lambda: (cur, ascending_or_valley(prev, cur)), lambda: (cur, descending_or_peak(prev, cur)))

def ascending_or_valley(prev, cur):
    return tf.cond(tf.logical_or(tf.equal(prev[1], 'A'), tf.equal(prev[1], 'V')), lambda: np.array('A'), lambda: np.array('V'))

def descending_or_peak(prev, cur):
    return tf.cond(tf.logical_or(tf.equal(prev[1], 'A'), tf.equal(prev[1], 'V')), lambda: np.array('P'), lambda: np.array('D'))

def label_local_extrema(tens):
    """Return a vector of chars indicating ascending, descending, peak, or valley slopes"""
    # initializer element values don't matter, just the type.
    initializer = (np.array(0, dtype=np.float32), np.array('A'))
    # First, zero out the trailing side
    slope = tf.scan(get_slope, tens, initializer)
    # Next, let's make the leading side the trailing side
    # shift by one, since each slope indicator is the slope
    # of the previous node (necessary to identify peaks and valleys)
    return slope[1][1:]


def find_local_maxima(tens):
    """Return the indices of the local maxima of the first dimension of the tensor"""
    return tf.squeeze(tf.where(tf.equal(label_local_extrema(blur_x_tf), 'P')))

# Rank of the blur tensors is unknown, so set it to 1 here for 1d skeletonization
blur_x_tf.set_shape([None])
blur_y_tf.set_shape([None])

# Find points on skeletonized arrays (where returns 1-length tuple)
lines_x = find_local_maxima(blur_x_tf) # vertical lines
lines_y = find_local_maxima(blur_y_tf) # horizontal lines

def numpy_diff(tensor, n=1):
    if n == 0:
        return tensor
    return numpy_diff(tensor[1:] - tensor[:-1], n-1)

def pruneLines(lineset):
    """Prunes a set of lines to 7 in consistent increasing order (chessboard)"""
    # This means we want the 7 consecutive elements whose 2nd order 
    # differential is between -5 and 5
    # TODO: 2nd order differential allows gradual shift (ie 45, 49,53,57)
    # which is undesirable.  Find a better abstraction for consistency 
    # such as max(n) - min(n) < 9
    diff = numpy_diff(lineset, 2)
    abs_diff = tf.abs(diff)
    
    # shortcut: if we have only 7 lines, and max<5, return all 7
    # otherwise, continue on with 2nd half of function.
    return tf.cond(tf.reduce_all(abs_diff<5), lambda: lineset, lambda: pruneLines2(lineset, abs_diff))
    
def pruneLines2(lineset, abs_diff):
    # count consecutive values < 5
    l = lambda prev, cur: tf.cond(cur<5, lambda: prev+1, lambda: tf.zeros_like(prev))
    initializer = np.array(0, dtype=np.int64)
    consec_running_count = tf.scan(l, abs_diff, initializer)
    # Find the first occurrence of 5 in the running count (representing 
    # the first set of seven roughly evenly spaced lines)
    first_five = tf.cast(tf.where(tf.equal(consec_running_count, 5)), tf.int32)
    # TODO: figure out what to do when we find no suitable result
#     return tf.cond(tf.equal(first_five.shape, [0,1]), lambda: first_five, lambda: tf.slice(lineset, first_five[0]-4, [7]))
    return tf.slice(lineset, first_five[0]-4, [7])

# Prune inconsistent lines
p_lines_x = pruneLines(lines_x)
p_lines_y = pruneLines(lines_y)

# TODO: handle failed pruning

# end getChessLines
# getChessTiles(img_arr, lines_x, lines_y)


In [50]:
stepx = tf.cast(tf.reduce_mean(numpy_diff(p_lines_x)), dtype=tf.int64)
stepy = tf.cast(tf.reduce_mean(numpy_diff(p_lines_y)), dtype=tf.int64)

print(p_lines_x)
# stepx.set_shape([1])
# stepy.set_shape([1])
p_lines_x.set_shape([7])

left_pad = tf.subtract(p_lines_x[0], stepx)
img_shape = tf.shape(img_gray, out_type=tf.int64)
right_pad = tf.subtract(img_shape[0], tf.add(p_lines_x[6], stepx))
top_pad = tf.subtract(p_lines_y[0], stepy)
bottom_pad = tf.subtract(img_shape[1], tf.add(p_lines_y[6], stepy))
pad = tf.stack([top_pad, bottom_pad,left_pad,right_pad])
pad_pos = tf.map_fn(lambda x: tf.cond(x>0, lambda: tf.zeros_like(x), lambda: x*-1), pad)

padded_img = tf.pad(img_gray, tf.stack([pad_pos[0], pad_pos[1]],[pad_pos[2], pad_pos[3]]), "SYMMETRIC")
with tf.Session() as sess:
#     t = tf.constant([[1, 2, 3], [4, 5, 6]])
#     paddings = tf.constant([[1, 1,], [2, 2]])
#     s = tf.get_default_graph().get_tensor_by_name("shrunkenimage:0")
#     print(s.shape)
    # 'constant_values' is 0.
    # rank of 't' is 2.
#     p = tf.pad(t, paddings, "CONSTANT") 
    print(pad.shape)
    pad_res = sess.run([pad_pos])
    print(pad_res[0])
    print(pad_res[0].shape)

Tensor("cond_2/Merge:0", shape=(7,), dtype=int64)


TypeError: '<' not supported between instances of 'list' and 'int'

In [ ]:
def getChessTiles(a, lines_x, lines_y):
  """Split up input grayscale array into 64 tiles stacked in a 3D matrix using the chess linesets"""
  # Find average square size, round to a whole pixel for determining edge pieces sizes

  stepx = np.int32(np.round(np.mean(np.diff(lines_x))))
  stepy = np.int32(np.round(np.mean(np.diff(lines_y))))
  
  # Pad edges as needed to fill out chessboard (for images that are partially over-cropped)
  padr_x = 0
  padl_x = 0
  padr_y = 0
  padl_y = 0
  
  if lines_x[0] - stepx < 0:
    padl_x = np.abs(lines_x[0] - stepx)
  if lines_x[-1] + stepx > a.shape[1]-1:
    padr_x = np.abs(lines_x[-1] + stepx - a.shape[1])
  if lines_y[0] - stepy < 0:
    padl_y = np.abs(lines_y[0] - stepy)
  if lines_y[-1] + stepx > a.shape[0]-1:
    padr_y = np.abs(lines_y[-1] + stepy - a.shape[0])
  
  # New padded array
  a2 = np.pad(a, ((padl_y,padr_y),(padl_x,padr_x)), mode='edge')
  
  setsx = np.hstack([lines_x[0]-stepx, lines_x, lines_x[-1]+stepx]) + padl_x
  setsy = np.hstack([lines_y[0]-stepy, lines_y, lines_y[-1]+stepy]) + padl_y
  
  a2 = a2[setsy[0]:setsy[-1], setsx[0]:setsx[-1]]
  setsx -= setsx[0]
  setsy -= setsy[0]

  # Tiles will contain 32x32x64 values corresponding to 64 chess tile images
  # A resize is needed to do this
  # tiles = np.zeros([np.round(stepy), np.round(stepx), 64],dtype=np.uint8)
  tiles = np.zeros([32, 32, 64],dtype=np.float32)
  
  # For each row
  for i in range(0,8):
    # For each column
    for j in range(0,8):
      # Vertical lines
      x1 = setsx[i]
      x2 = setsx[i+1]
      padr_x = 0
      padl_x = 0
      padr_y = 0
      padl_y = 0

      if (x2-x1) > stepx:
        if i == 7:
          x1 = x2 - stepx
        else:
          x2 = x1 + stepx
      elif (x2-x1) < stepx:
        if i == 7:
          # right side, pad right
          padr_x = stepx-(x2-x1)
        else:
          # left side, pad left
          padl_x = stepx-(x2-x1)
      # Horizontal lines
      y1 = setsy[j]
      y2 = setsy[j+1]

      if (y2-y1) > stepy:
        if j == 7:
          y1 = y2 - stepy
        else:
          y2 = y1 + stepy
      elif (y2-y1) < stepy:
        if j == 7:
          # right side, pad right
          padr_y = stepy-(y2-y1)
        else:
          # left side, pad left
          padl_y = stepy-(y2-y1)
      # slicing a, rows sliced with horizontal lines, cols by vertical lines so reversed
      # Also, change order so its A1,B1...H8 for a white-aligned board
      # Apply padding as defined previously to fit minor pixel offsets
      # tiles[:,:,(7-j)*8+i] = np.pad(a2[y1:y2, x1:x2],((padl_y,padr_y),(padl_x,padr_x)), mode='edge')
      full_size_tile = np.pad(a2[y1:y2, x1:x2],((padl_y,padr_y),(padl_x,padr_x)), mode='edge')
      tiles[:,:,(7-j)*8+i] = np.asarray( \
          full_size_tile, dtype=np.float32) / 255.0
#         PIL.Image.fromarray(full_size_tile) \
#         .resize([32,32], PIL.Image.BILINEAR), dtype=np.float32) / 255.0
        #PIL.Image.ADAPTIVE causes image artifacts
  return tiles

In [53]:
import numpy as np
import tensorflow as tf

X_node    = tf.placeholder('float',[1,10,1])
filter_tf = tf.Variable( tf.truncated_normal([3,1,1],stddev=0.1) )

Xconv_tf_tensor = tf.nn.conv1d(X_node, filter_tf,1,'SAME')

X = np.random.normal(0,1,[1,10,1])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    feed_dict = {X_node: X}
    filter_np = filter_tf.eval()
    Xconv_tf = sess.run(Xconv_tf_tensor,feed_dict)
    Xconv_np = np.convolve(X[0,:,0],filter_np[:,0,0],'SAME')
    filter_np2=filter_np[::-1,0,0]
    print(filter_np)
    print(filter_np2)
    np.allclose(np.convolve(X[0,:,0],filter_np2,'SAME'),  Xconv_tf.flatten())
    np.convolve(X[0,:,0],filter_np2,'SAME')

[[[ 0.12066936]]

 [[-0.06092268]]

 [[-0.05091505]]]
[-0.05091505 -0.06092268  0.12066936]


In [69]:
import scipy.signal
gausswin = scipy.signal.gaussian(21,4)
gausswin /= np.sum(gausswin)
print(gausswin)

[ 0.00441961  0.00800287  0.01361335  0.02175407  0.03265673  0.04605336
  0.0610108   0.07592916  0.08877018  0.09749497  0.1005898   0.09749497
  0.08877018  0.07592916  0.0610108   0.04605336  0.03265673  0.02175407
  0.01361335  0.00800287  0.00441961]
